# Evaluate Plots V8

json을 통해서 읽는다.
0613-morning: jch v8 half for preliminary analysis

In [5]:
# %matplotlib qt
import sys
import os
import numpy as np
import json
import torch
import matplotlib.pyplot as plt
import mplcursors
from sklearn.manifold import TSNE
from sentence_transformers import SentenceTransformer
from fastdtw import fastdtw
from scipy.spatial.distance import cosine
from bert_score import score
import pickle

#soft dtw
import torch
from tslearn.metrics import soft_dtw

sys.path.append(os.path.abspath('/root/project')) # add root path to sys.path
sys.path.append(os.path.abspath('/usr/local/lib/python3.10/dist-packages'))
from util import util_constants
from util import util_funcs
import f4_evaluate.evaluate_scene as evaluate_scene
import f1_init.database_init as database_init
import f1_init.agent_init as agent_init
import f1_init.constants_init as constants_init

#Computing similarity
import json
from sentence_transformers import SentenceTransformer
from bert_score import score
import numpy as np
from numpy.linalg import norm
import ast
import pandas as pd


#BASELINE FROM v8/ index range(300) from jch 
PATH_CURR_FOLDER = os.path.abspath('') 

path_b0_p1 = os.path.abspath(os.path.join(PATH_CURR_FOLDER, 'result_v8/ispossible/base0-part1-300.json'))
path_b0_p2 = os.path.abspath(os.path.join(PATH_CURR_FOLDER, 'result_v8/ispossible/base0-part2-300.json'))

path_b1_1_p1 = os.path.abspath(os.path.join(PATH_CURR_FOLDER, 'result_v8/ispossible/base1-1-part1-300.json'))
path_b1_1_p2 = os.path.abspath(os.path.join(PATH_CURR_FOLDER, 'result_v8/ispossible/base1-1-part2-300.json'))

path_b1_2_p1 = os.path.abspath(os.path.join(PATH_CURR_FOLDER, 'result_v8/ispossible/base1-2-part1-300.json'))
path_b1_2_p2 = os.path.abspath(os.path.join(PATH_CURR_FOLDER, 'result_v8/ispossible/base1-2-part2-300.json'))

path_b2_1_p1 = os.path.abspath(os.path.join(PATH_CURR_FOLDER, 'result_v8/ispossible/base2-1-part1-300.json'))
path_b2_1_p2 = os.path.abspath(os.path.join(PATH_CURR_FOLDER, 'result_v8/ispossible/base2-1-part2-300.json'))

path_b2_2_p1 = os.path.abspath(os.path.join(PATH_CURR_FOLDER, 'result_v8/ispossible/base2-2-part1-300.json'))
path_b2_2_p2 = os.path.abspath(os.path.join(PATH_CURR_FOLDER, 'result_v8/ispossible/base2-2-part2-300.json'))



In [6]:
def load_pickle_file(path):
    try:          
         with open(path, "rb") as f:
             data = pickle.load(f)
             return data
    except:
         return None

def load_json_file(path):
    try:
        with open(path, 'r') as f:
            data = json.load(f)
            return data
    except:
        return None

def load_json_utf8(path):
    with open(path, 'r', encoding='utf-8') as f:
        dict = json.load(path)
    return dict


def check_pickle_file(path):
    try:
        with open(path, "rb") as f:
            pickle.load(f)  # try loading to ensure file is not empty or corrupted
        return True
    except (EOFError, FileNotFoundError, PermissionError, IsADirectoryError, pickle.UnpicklingError):
        return False

def check_json_file(path):
    try:
        with open(path, "rb") as f:
            json.load(f)  # try loading to ensure file is not empty or corrupted
        return True
    except (EOFError, FileNotFoundError, PermissionError, IsADirectoryError, pickle.UnpicklingError):
        return False   

def normalize_to_string(value):
    '''
    used so to cope with value from a dictionary that happens to be inside a list bracket    
    '''
    # If it's a list and the first element is a string, return that
    if isinstance(value, list) and len(value) > 0 and isinstance(value[0], str):
        return value[0]
    # If it's already a string, return it
    elif isinstance(value, str):
        return value
    # Fallback for anything else
    else:
        return str(value)
    



In [7]:
# %matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

## v8_evaluator Plots

evaluator_1: (core)  
evaluator_2: (entity)(core)  
evaluator_3: (entity)(sim)(core)  
evaluator_h: (entity)(sim)(human)  

In [ ]:
path_eval_results = constants_init.PATH_ROOT + "f4_evaluate/result_v6/"
baseline_folders = ['output-rag-0602','output-norag-0602','output-1goalmediation-0602','output-1goalmediation-norag-0602']

# #---------------------------------------
# # read human annotated list
# #---------------------------------------
import json

data_base0_p1 = load_json_file(path_b0_p1)
data_base0_p2 = load_json_file(path_b0_p2)

data_base1_1_p1 = load_json_file(path_b1_1_p1)
data_base1_1_p2 = load_json_file(path_b1_1_p2)
data_base1_2_p1 = load_json_file(path_b1_2_p1)
data_base1_2_p2 = load_json_file(path_b1_2_p2)

data_base2_1_p1 = load_json_file(path_b2_1_p1)
data_base2_1_p2 = load_json_file(path_b2_1_p2)
data_base2_2_p1 = load_json_file(path_b2_2_p1)
data_base2_2_p2 = load_json_file(path_b2_2_p2)

#.sort_values(by='idx').reset_index(drop=True)
df_base0 = pd.concat([pd.DataFrame(data_base0_p1)  , pd.DataFrame(data_base0_p2)  ], ignore_index=True)
df_base1_1 = pd.concat([pd.DataFrame(data_base1_1_p1)  , pd.DataFrame(data_base1_1_p2)  ], ignore_index=True)
df_base1_2 = pd.concat([pd.DataFrame(data_base1_2_p1)  , pd.DataFrame(data_base1_2_p2)  ], ignore_index=True)
df_base2_1 = pd.concat([pd.DataFrame(data_base2_1_p1)  , pd.DataFrame(data_base2_1_p2)  ], ignore_index=True)
df_base2_2 = pd.concat([pd.DataFrame(data_base2_2_p1)  , pd.DataFrame(data_base2_2_p2)  ], ignore_index=True)
                    

# df_base0 = pd.DataFrame()  
# df_base0 = pd.DataFrame(data_base0)  
# df_base1_norag = pd.DataFrame(data_base1_norag)
# df_base1_rag = pd.DataFrame(data_base1_rag)
# df_base2_norag_p1 = pd.DataFrame(data_base2_norag_p1)
# df_base2_rag_p1 = pd.DataFrame(data_base2_rag_p1)
# df_base2_norag_p2 = pd.DataFrame(data_base2_norag_p2)
# df_base2_rag_p2 = pd.DataFrame(data_base2_rag_p2)



#[
# 'idx', 'baseline', 'source_idx', 'source_uid', 'target_uid',
#  'target_equal_ratio', 'trial_idx', 
# 'source_file_name', 'target_file_name', 
# 'source_action_sequence', 'source_scene_graph', 'target_scene_graph',
#  'source_goal_category', 'source_goal_description', 
# 'core_activity', 'target_sequence', 
# 'source_taxonomy', 'target_taxonomy', 
# 'human_entitycheck_tax', 'human_entitycheck_seq', 
# 'human_passcheck_tax', 'human_passcheck_seq', 
# 'human_coreactivitycheck', 
# 'isPossible', 
# 'edited'
#]

# print("dtype: seq, seq, tax, tax")
# #source sequence is a list of characters
# print(df_base0['source_action_sequence'].dtype)
# print(df_base0['target_sequence'].dtype)
# print(df_base0['source_taxonomy'].dtype)
# print(df_base0['target_taxonomy'].dtype)

# print("dtype: entitytax, entityseq, passtax, passseq, passcore, ispossible")
# print(df_base0['human_entitycheck_tax'].dtype)
# print(df_base0['human_entitycheck_seq'].dtype)
# print(df_base0['human_passcheck_tax'].dtype)
# print(df_base0['human_passcheck_seq'].dtype)
# print(df_base0['human_coreactivitycheck'].dtype)
# print(df_base0['isPossible'].dtype)



['idx', 'baseline', 'source_idx', 'source_uid', 'target_uid', 'target_equal_ratio', 'trial_idx', 'source_file_name', 'target_file_name', 'source_action_sequence', 'source_scene_graph', 'target_scene_graph', 'source_goal_category', 'source_goal_description', 'core_activity', 'target_sequence', 'human_entitycheck_tax', 'human_entitycheck_seq', 'human_coreactivitycheck', 'human_passcheck_tax', 'human_passcheck_seq', 'source_taxonomy', 'target_taxonomy', 'isPossible', 'edited']
dtype: seq, seq, tax, tax
object
object
object
object
dtype: entitytax, entityseq, passtax, passseq, passcore, ispossible
bool
bool
bool
bool
bool
bool


In [ ]:
#========================================
# seq (df object => list of double quoted strings) : read_df_seqence()
#========================================
def convert_df_sequence(sequence):
    '''
    converts sequences from df to iterable list of strings for similarity measurement later.

    args:
        sequence: object datatype from dataframe
    
    return
        list of actions, where each action is enclosed in double quotes
    '''
    if isinstance(sequence, list):
        return [str(x).strip().strip('"') for x in sequence]
    elif isinstance(sequence, str):
        # Fall back for stringified list (just in case)
        try:
            import ast
            parsed = ast.literal_eval(sequence)
            if isinstance(parsed, list):
                return [str(x).strip().strip('"') for x in parsed]
        except:
            pass
        # fallback to comma split
        return [s.strip().strip('"') for s in sequence.split(',')]
    else:
        raise ValueError(f"Unsupported type: {type(sequence)}")

#========================================
# tax (df object => list of two dictionaries) : json.loads()
#========================================
# source_tax = df_base2_rag.loc[4, 'source_taxonomy']
# target_tax = df_base2_rag.loc[4, 'target_taxonomy']
# source_tax = json.loads(source_tax)
# target_tax = json.loads(target_tax)
def apply_json_loads(tax):
    return json.loads(tax)


#========================================
# Apply to our dataframes(only once after dataset loading)
#========================================
def apply_seq_conversion_to_df(df:pd.DataFrame):
    df['source_action_sequence'] = df['source_action_sequence'].apply(convert_df_sequence)
    df['target_sequence'] = df['target_sequence'].apply(convert_df_sequence)

def apply_tax_conversion_to_df(df:pd.DataFrame):
    df['source_taxonomy'] = df['source_taxonomy'].apply(apply_json_loads)
    df['target_taxonomy'] = df['target_taxonomy'].apply(apply_json_loads)

apply_seq_conversion_to_df(df_base0)
apply_seq_conversion_to_df(df_base1_1)
apply_seq_conversion_to_df(df_base1_2)
apply_seq_conversion_to_df(df_base2_1)
apply_seq_conversion_to_df(df_base2_2)

apply_tax_conversion_to_df(df_base2_1)
apply_tax_conversion_to_df(df_base2_2)

print(df_base0['source_action_sequence'].dtype)
print(df_base0['target_sequence'].dtype)
print(df_base0['source_taxonomy'].dtype)
print(df_base0['target_taxonomy'].dtype)


sequence only baseline. no taxonomy
object
object
object
object


In [ ]:
#========================================
# check iterations and sizes of sequence and tax
#========================================
source_seq = df_base2_2.loc[4, 'source_action_sequence']
target_seq = df_base2_2.loc[4, 'target_sequence']

print(source_seq)
print(target_seq)

for item in source_seq:
    print(item)

for item in target_seq:
    print(item)

source_tax = df_base2_2.loc[4, 'source_taxonomy']
target_tax = df_base2_2.loc[4, 'target_taxonomy']

for item in source_tax:
    print(item)

for item in target_tax:
    print(item)

#========================================
# dtw between df columns
#========================================
import openai
import numpy as np
from fastdtw import fastdtw

openai.api_key = os.getenv("OPENAI_API_KEY")  

def embed_texts(text_list, model="text-embedding-3-large"):
    """Generate embeddings for a list of text strings."""
    response = openai.embeddings.create(
        model=model,
        input=text_list
    )
    return np.array([d.embedding for d in response.data])

def compute_dtw_distance(seq1, seq2):
    """Compute DTW distance between two sequences of text using embeddings."""
    if not isinstance(seq1, list) or not isinstance(seq2, list) or len(seq1) == 0 or len(seq2) == 0:
        return np.nan

    embeddings1 = embed_texts(seq1)
    embeddings2 = embed_texts(seq2)

    distance, path = fastdtw(embeddings1, embeddings2, dist=cosine)
    similarity = 1 - (distance / len(path))
    return similarity

def compute_dtw_column(df):
    """Compute DTW for each row of the DataFrame and store it in a new column."""
    dtw_results = []
    for i, row in df.iterrows():
        try:
            print(row['source_action_sequence'])
            print(row['target_sequence'])
            dtw_val = compute_dtw_distance(row['source_action_sequence'], row['target_sequence'])
            print(dtw_val)
        except Exception as e:
            print(f"Error on row {i}: {e}")
            dtw_val = np.nan
        dtw_results.append(dtw_val)
    df['dtw_similarity'] = dtw_results
    return df


# apply_seq_conversion_to_df(df_base0)
# apply_seq_conversion_to_df(df_base1_1)
# apply_seq_conversion_to_df(df_base1_2)
# apply_seq_conversion_to_df(df_base2_1)
# apply_seq_conversion_to_df(df_base2_2)

# apply_tax_conversion_to_df(df_base2_1)
# apply_tax_conversion_to_df(df_base2_2)

print(df_base0['source_action_sequence'].dtype)
print(df_base0['target_sequence'].dtype)
print(df_base0['source_taxonomy'].dtype)
print(df_base0['target_taxonomy'].dtype)

# df_base0 = compute_dtw_column(df_base0)
# df_base1_1 = compute_dtw_column(df_base1_1)
# df_base1_2 = compute_dtw_column(df_base1_2)
df_base2_1 = compute_dtw_column(df_base2_1)
df_base2_2 = compute_dtw_column(df_base2_2)

df_base0.to_pickle("df_base0.pkl")
df_base1_1.to_pickle("df_base1_1.pkl")
df_base1_2.to_pickle("df_base1_2.pkl")
df_base2_1.to_pickle("df_base2_1.pkl")
df_base2_2.to_pickle("df_base2_2.pkl")

# df_base0_dt = compute_dtw_column(df_base0)
# df_base1_norag_dt = compute_dtw_column(df_base1_norag)
# df_base1_rag_dt = compute_dtw_column(df_base1_rag)
# df_base2_norag_dt_p1 = compute_dtw_column(df_base2_norag_p1)
# df_base2_rag_dt_p1 = compute_dtw_column(df_base2_rag_p1)
# df_base2_norag_dt_p2 = compute_dtw_column(df_base2_norag_p2)
# df_base2_rag_dt_p2 = compute_dtw_column(df_base2_rag_p2)

# df_base0_dt.to_pickle("df_base0_dtw.pkl")
# df_base1_norag_dt.to_pickle("df_base1_norag_dtw.pkl")
# df_base1_rag_dt.to_pickle("df_base1_rag_dtw.pkl")
# df_base2_norag_dt_p1.to_pickle("df_base2_norag_p1_dtw.pkl")
# df_base2_rag_dt_p1.to_pickle("df_base2_rag_p1_dtw.pkl")
# df_base2_norag_dt_p2.to_pickle("df_base2_norag_p2_dtw.pkl")
# df_base2_rag_dt_p2.to_pickle("df_base2_rag_p2_dtw.pkl")

['Arrange cloth', 'Arrange cloth', 'Arrange cloth', 'Arrange cloth', 'Fold towel', 'Arrange cloth', 'Fold handkerchief', 'Pack hanger']
['fold towel', 'fold handkerchief', 'fold cloth', 'arrange towel on plate', 'arrange handkerchief on plate', 'arrange cloth on plate']
Arrange cloth
Arrange cloth
Arrange cloth
Arrange cloth
Fold towel
Arrange cloth
Fold handkerchief
Pack hanger
fold towel
fold handkerchief
fold cloth
arrange towel on plate
arrange handkerchief on plate
arrange cloth on plate
{'arranging surface': 'clothes horse', 'arranging action': 'folding'}
{'cloth type': 'towel', 'cloth container': 'clothes horse'}
{'arranging surface': 'plate', 'arranging action': 'folding'}
{'cloth type': 'towel', 'cloth container': 'plate'}
['Cut the mango', 'Add chopped mango into the pan', 'slice mango', 'add slice mango into the pot', 'cut the mango', 'add mango to pot', 'cut the mango', 'add cut mango to pot', 'cut the mango', 'add ingredient to pot', 'cut mango', 'add mango to pan', 'cut m

In [16]:
# df_base0_dt = load_pickle_file("df_base0_dtw.pkl")
# df_base1_norag_dt= load_pickle_file("df_base1_norag_dtw.pkl")
# df_base1_rag_dt= load_pickle_file("df_base1_rag_dtw.pkl")
# df_base2_norag_dt_p1= load_pickle_file("df_base2_norag_p1_dtw.pkl")
# df_base2_rag_dt_p1= load_pickle_file("df_base2_rag_p1_dtw.pkl")
# df_base2_norag_dt_p2= load_pickle_file("df_base2_norag_p2_dtw.pkl")
# df_base2_rag_dt_p2= load_pickle_file("df_base2_rag_p2_dtw.pkl")

# num_bools0 = df_base0_dt["isPossible"].eq(True).sum()
# num_bools1_1 = df_base1_norag_dt["isPossible"].eq(True).sum()
# num_bools1_2 = df_base1_rag_dt["isPossible"].eq(True).sum()
# num_bools2_1_p1 = df_base2_norag_dt_p1["isPossible"].eq(True).sum()
# num_bools2_2_p1 = df_base2_rag_dt_p1["isPossible"].eq(True).sum()
# num_bools2_1_p2 = df_base2_norag_dt_p2["isPossible"].eq(True).sum()
# num_bools2_2_p2 = df_base2_rag_dt_p2["isPossible"].eq(True).sum()

# yield_0 = df_base0_dt[(df_base0_dt['isPossible'] == True) & (df_base0_dt['human_passcheck_seq'] == True)]
# yield_1_1 = df_base1_norag_dt[(df_base1_norag_dt['isPossible'] == True) & (df_base1_norag_dt['human_passcheck_seq'] == True)]
# yield_1_2 = df_base1_rag_dt[(df_base1_rag_dt['isPossible'] == True) & (df_base1_rag_dt['human_passcheck_seq'] == True)]
# yield_2_1_p1 = df_base2_norag_dt_p1[(df_base2_norag_dt_p1['isPossible'] == True) & (df_base2_norag_dt_p1['human_passcheck_seq'] == True)]
# yield_2_2_p1 = df_base2_rag_dt_p1[(df_base2_rag_dt_p1['isPossible'] == True) & (df_base2_rag_dt_p1['human_passcheck_seq'] == True)]
# yield_2_1_p2 = df_base2_norag_dt_p2[(df_base2_norag_dt_p2['isPossible'] == True) & (df_base2_norag_dt_p2['human_passcheck_seq'] == True)]
# yield_2_2_p2 = df_base2_rag_dt_p2[(df_base2_rag_dt_p2['isPossible'] == True) & (df_base2_rag_dt_p2['human_passcheck_seq'] == True)]

# print("ISPOSSIBLE")
# print(f"{num_bools0} {num_bools1_1} {num_bools1_2}")
# print(f"base2 pt1 pt1 pt2 pt2 {num_bools2_1_p1} {num_bools2_2_p1} {num_bools2_1_p2} {num_bools2_2_p2}")
# print(f"base2 total: {num_bools2_1_p1+num_bools2_1_p2} {num_bools2_2_p1+num_bools2_2_p2}") 

# print("YIELD")
# print(f"{yield_0.shape[0]} {yield_1_1.shape[0]} {yield_1_2.shape[0]}")
# print(f"pt1 pt1 pt2 pt2 {yield_2_1_p1.shape[0]} {yield_2_2_p1.shape[0]} {yield_2_1_p2.shape[0]} {yield_2_2_p2.shape[0]}")
# print(f"{yield_2_1_p1.shape[0]+yield_2_1_p2.shape[0]} {yield_2_2_p1.shape[0]+yield_2_2_p2.shape[0]}")

# print("YIELD / ISPOSSIBLE")
# print(f"{yield_0.shape[0]/num_bools0:.2f} {yield_1_1.shape[0]/num_bools1_1:.2f} {yield_1_2.shape[0]/num_bools1_2:.2f}")
# print(f"{yield_2_1_p1.shape[0]/num_bools2_1_p1:.2f} {yield_2_2_p1.shape[0]/num_bools2_2_p1:.2f} {yield_2_1_p2.shape[0]/num_bools2_1_p2:.2f} {yield_2_2_p2.shape[0]/num_bools2_2_p2:.2f}")


# # for 0-299 idx 0th:
# # is possible = 171 175 195 163 184
# # seq yield in ispossible = 154 165 181 159 183
# pd.set_option('display.max_rows', None)  # Show all rows
# print(yield_2_2_p1['core_activity'])
# print(yield_2_2_p2['core_activity'])



df_base0 = load_pickle_file("df_base0.pkl")
df_base1_1 = load_pickle_file("df_base1_1.pkl")
df_base1_2 = load_pickle_file("df_base1_2.pkl")
df_base2_1 = load_pickle_file("df_base2_1.pkl")
df_base2_2 = load_pickle_file("df_base2_2.pkl")

num_bools0 = df_base0["isPossible"].eq(True).sum()
num_bools1_1 = df_base1_1["isPossible"].eq(True).sum()
num_bools1_2 = df_base1_2["isPossible"].eq(True).sum()
num_bools2_1 = df_base2_1["isPossible"].eq(True).sum()
num_bools2_2 = df_base2_2["isPossible"].eq(True).sum()

yield_0 = df_base0[(df_base0['isPossible'] == True) & (df_base0['human_passcheck_seq'] == True)]
yield_1_1 = df_base1_1[(df_base1_1['isPossible'] == True) & (df_base1_1['human_passcheck_seq'] == True)]
yield_1_2 = df_base1_2[(df_base1_2['isPossible'] == True) & (df_base1_2['human_passcheck_seq'] == True)]
yield_2_1= df_base2_1[(df_base2_1['isPossible'] == True) & (df_base2_1['human_passcheck_seq'] == True)]
yield_2_2 = df_base2_2[(df_base2_2['isPossible'] == True) & (df_base2_2['human_passcheck_seq'] == True)]

print(f"{num_bools0} {num_bools1_1} {num_bools1_2}")
print(f"base2_1 2_2 {num_bools2_1} {num_bools2_2}")

print("YIELD")
print(f"{yield_0.shape[0]} {yield_1_1.shape[0]} {yield_1_2.shape[0]}")
print(f"pt1 pt1 pt2 pt2 {yield_2_1.shape[0]} {yield_2_2.shape[0]}")

print("YIELD / ISPOSSIBLE")
print(f"{yield_0.shape[0]/num_bools0:.2f} {yield_1_1.shape[0]/num_bools1_1:.2f} {yield_1_2.shape[0]/num_bools1_2:.2f}")
print(f"{yield_2_1.shape[0]/num_bools2_1:.2f} {yield_2_2.shape[0]/num_bools2_2:.2f}")


323 315 347
base2_1 2_2 313 328
YIELD
292 297 328
pt1 pt1 pt2 pt2 299 318
YIELD / ISPOSSIBLE
0.90 0.94 0.95
0.96 0.97


## Plot
### target_target_equal_ratio VS ACCEPTANCE RATE
### target_target_equal_ratio vs ispossible RATIO 


- 'idx', 'baseline', 'source_idx', 'source_uid', 'target_uid',
-  'target_equal_ratio', 'trial_idx', 
- 'source_file_name', 'target_file_name', 
- 'source_action_sequence', 'source_scene_graph', 'target_scene_graph',
-  'source_goal_category', 'source_goal_description', 
- 'core_activity', 'target_sequence', 
- 'source_taxonomy', 'target_taxonomy', 
- 'human_entitycheck_tax', 'human_entitycheck_seq', 
- 'human_passcheck_tax', 'human_passcheck_seq', 
- 'human_coreactivitycheck', 
- 'isPossible', 
- 'edited'

In [ ]:
#==================================
# 1. target_equal_ratio vs ACCEPTANCE RATE
#==================================



#==================================
# 2. target_equal_ratio vs ispossible RATIO
#==================================





In [17]:
#-------------------------------------
# SELECTION RATIO FUNCTIONS
#-------------------------------------
#HUMAN EVALUATOR가 가장 이상적이라고 가정할 때에 다른 evaluator에 대해서 precision-recall curve를 그릴 수 있다.
#PRECISION = TP / (TP+FP)예측한 positive
#RECALL = TP / (TP+FN)실제 positive
#-------------------------------------
# OLDER PLOTS with scatters
#-------------------------------------
def plot_sequence_cursor_interactive(df: pd.DataFrame, mybool: bool, key2plot: str, bin_num:int):
    # df = df.dropna(subset=['target_equal_ratio', 'sequence_similarity_sbert', 'sequence_boolean'])
    filtered_df = df[df['sequence_boolean'] == mybool]
    size1 = df.shape[0]
    size2 = filtered_df.shape[0]
    print(size1, size2)

    #===============================
    #ratio of selection
    #===============================
    # Step 1: Bin the data by target_equal_ratio
    bin_width = 1.0/bin_num
    # bins = np.arange(0, 1.0001 + bin_width, bin_width)
    bins = np.arange(0, 0.9001 + bin_width, bin_width)
    df['inclusion_bin'] = pd.cut(df['target_equal_ratio'], bins=bins, include_lowest=True)

    # Step 2: Group by bins and calculate ratio
    grouped = df.groupby('inclusion_bin')
    bin_centers = [interval.mid for interval in grouped.groups.keys()]

    ratios = []
    for bin_label, group in grouped:
        total = len(group)
        true_count = group[group['sequence_boolean'] == True].shape[0]
        ratio = true_count / total if total > 0 else 0
        ratios.append(ratio)


    # Scatter plot with hover info
    fig = px.scatter(
        filtered_df,
        x='target_equal_ratio',
        y=key2plot,
        hover_data=['idx','source_uid'],
        labels={
            'target_equal_ratio': 'Inclusion Ratio',
            key2plot: key2plot,
        },
        title=f'Inclusion Ratio vs. {key2plot} (bool={mybool}) {size2}/{size1}',
    )

    fig.update_layout(
        xaxis=dict(range=[-0.1, 1.1]),
        yaxis=dict(range=[0, 1.1]),
    )

    #===============================
    #ratio of selection
    #===============================
    # Add ratio line trace with secondary y-axis
    fig.add_trace(go.Scatter(
        x=bin_centers,
        y=ratios,
        mode='lines+markers',
        name='Ratio (mybool=True / all)',
        line=dict(color='green', dash='dot'),
        yaxis='y2'
    ))

    # Configure secondary y-axis
    fig.update_layout(
        yaxis2=dict(
            title='Ratio',
            overlaying='y',
            side='right',
            range=[0, 1.1]
        )
    )

    fig.show()

def plot_taxonomy_cursor_interactive(df: pd.DataFrame, togglebool: bool, mybool: bool, key2plot: str, bin_num:int):
    # Clean data
    df = df.dropna(subset=['target_equal_ratio', 'taxonomy_similarity_sbert', 'taxonomy_boolean', 'taxonomy_weighted_similarity'])
    size1 = df.shape[0]
    
    # Filter based on toggle
    if togglebool:
        filtered_df = df[df['taxonomy_boolean'] == mybool]
    else:
        filtered_df = df
    size2 = filtered_df.shape[0]

    #===============================
    #ratio of selection
    #===============================
    # Step 1: Bin the data by target_equal_ratio
    bin_width = 1.0/bin_num
    # bins = np.arange(0, 1.0001 + bin_width, bin_width)
    bins = np.arange(0, 0.9001 + bin_width, bin_width)
    df['inclusion_bin'] = pd.cut(df['target_equal_ratio'], bins=bins, include_lowest=True)

    # Step 2: Group by bins and calculate ratio
    grouped = df.groupby('inclusion_bin')
    bin_centers = [interval.mid for interval in grouped.groups.keys()]

    ratios = []
    for bin_label, group in grouped:
        total = len(group)
        true_count = group[group['taxonomy_boolean'] == True].shape[0]
        ratio = true_count / total if total > 0 else 0
        ratios.append(ratio)


    # Polynomial fit (linear)
    N = 1
    coeffs = np.polyfit(filtered_df['target_equal_ratio'], filtered_df[key2plot], deg=N)
    poly_func = np.poly1d(coeffs)
    x_smooth = np.linspace(filtered_df['target_equal_ratio'].min(), filtered_df['target_equal_ratio'].max(), 200)
    y_smooth = poly_func(x_smooth)

    # Plotly scatter plot
    fig = px.scatter(
        filtered_df,
        x='target_equal_ratiotio',
        y=key2plot,
        hover_data=['source_uid', 'idx'],
        labels={
            'target_equal_ratiotiotio': 'Inclusion Ratio',
            key2plot: 'Taxonomy Similarity'
        },
        title=f'Inclusion Ratio vs. {key2plot} (taxonomy_boolean={mybool}) {size2}/{size1}'
    )

    # Add polynomial fit line
    fig.add_scatter(
        x=x_smooth,
        y=y_smooth,
        mode='lines',
        name=f'{N}° polynomial fit',
        line=dict(color='red')
    )

    # Adjust axis and layout
    fig.update_layout(
        xaxis=dict(range=[-0.1, 1.1]),
        yaxis=dict(range=[0, 1.1]),
        height=600,
        width=800
    )

    #===============================
    #ratio of selection
    #===============================
    # Add ratio line trace with secondary y-axis
    fig.add_trace(go.Scatter(
        x=bin_centers,
        y=ratios,
        mode='lines+markers',
        name='Ratio (mybool=True / all)',
        line=dict(color='green', dash='dot'),
        yaxis='y2'
    ))

    # Configure secondary y-axis
    fig.update_layout(
        yaxis2=dict(
            title='Ratio',
            overlaying='y',
            side='right',
            range=[0, 1.1]
        )
    )

    fig.show()

def plot_sequence_cursor_interactive2(df: pd.DataFrame, mybool: bool, key2plot: str, bin_num:int):
    # df = df.dropna(subset=['target_equal_ratiotio', 'sequence_similarity_sbert', 'core_check_seq'])
    filtered_df = df[df['core_check_seq'] == mybool]
    size1 = df.shape[0]
    size2 = filtered_df.shape[0]
    print(size1, size2)

    #===============================
    #ratio of selection
    #===============================
    # Step 1: Bin the data by target_equal_ratiotio
    bin_width = 1.0/bin_num
    # bins = np.arange(0, 1.0001 + bin_width, bin_width)
    bins = np.arange(0, 0.9001 + bin_width, bin_width)
    df['inclusion_bin'] = pd.cut(df['target_equal_ratiotio'], bins=bins, include_lowest=True)

    # Step 2: Group by bins and calculate ratio
    grouped = df.groupby('inclusion_bin', observed=True)
    bin_centers = [interval.mid for interval in grouped.groups.keys()]

    ratios = []
    for bin_label, group in grouped:
        total = len(group)
        true_count = group[group['core_check_seq'] == True].shape[0]
        ratio = true_count / total if total > 0 else 0
        ratios.append(ratio)



    # Scatter plot with hover info
    fig = px.scatter(
        filtered_df,
        x='target_equal_ratiotio',
        y=key2plot,
        hover_data=['idx','source_uid'],
        labels={
            'target_equal_ratio': 'Inclusion Ratio',
            key2plot: key2plot,
        },
        title=f'Inclusion Ratio vs. {key2plot} (bool={mybool}) {size2}/{size1}',
    )

    fig.update_layout(
        xaxis=dict(range=[-0.1, 1.1]),
        yaxis=dict(range=[0, 1.1]),
    )

    #===============================
    #ratio of selection
    #===============================
    # Add ratio line trace with secondary y-axis
    fig.add_trace(go.Scatter(
        x=bin_centers,
        y=ratios,
        mode='lines+markers',
        name='Ratio (mybool=True / all)',
        line=dict(color='green', dash='dot'),
        yaxis='y2'
    ))

    # Configure secondary y-axis
    fig.update_layout(
        yaxis2=dict(
            title='Ratio',
            overlaying='y',
            side='right',
            range=[0, 1.1]
        )
    )

    fig.update_layout(
        title={
            'text': f'Inclusion Ratio vs. {key2plot} (bool={mybool}) {size2}/{size1}',
            'x': 0.5,  # Center title
            'xanchor': 'center'
        },
        xaxis=dict(range=[-0.1, 1.1]),
        yaxis=dict(range=[0, 1.1]),
        width=700,   # Adjust width to be closer to height
        height=700,  # Make height the same or slightly less/more
    )    

    fig.show()

#taxonomy_similarity_sbert
#taxonomy_weighted_similarity
# plot_taxonomy_cursor_interactive(df_rag, False, True, 'taxonomy_similarity_sbert', 9)
# plot_taxonomy_cursor_interactive(df_norag, False, True, 'taxonomy_similarity_sbert',9)
# plot_taxonomy_cursor_interactive(df_rag, False, True, 'taxonomy_weighted_similarity', 9)
# plot_taxonomy_cursor_interactive(df_norag, False, True, 'taxonomy_weighted_similarity',9)

# plot_sequence_cursor_interactive(df_rag, True, 'sequence_similarity_sbert', 9)
# plot_sequence_cursor_interactive(df_norag, True, 'sequence_similarity_sbert',9)
# plot_sequence_cursor_interactive(df_raggoal, True, 'sequence_similarity_sbert', 9)
# plot_sequence_cursor_interactive(df_noraggoal, True, 'sequence_similarity_sbert', 9)

# plot_sequence_cursor_interactive(df_rag, True, 'sequence_dtw_similarity', 9)
# plot_sequence_cursor_interactive(df_norag, True, 'sequence_dtw_similarity',9)
# plot_sequence_cursor_interactive(df_raggoal, True, 'sequence_dtw_similarity', 9)
# plot_sequence_cursor_interactive(df_noraggoal, True, 'sequence_dtw_similarity', 9)

# plot_sequence_cursor_interactive2(df_eval_rag, True, 'sim_tax3', 5)
# plot_sequence_cursor_interactive2(df_eval_norag, True, 'sim_tax3', 5)
# plot_sequence_cursor_interactive2(df_eval_rag, True, 'sim_seq3', 5)
# plot_sequence_cursor_interactive2(df_eval_norag, True, 'sim_seq3', 5)
# plot_sequence_cursor_interactive2(df_eval_raggoal, True, 'sim_seq3', 5)
# plot_sequence_cursor_interactive2(df_eval_noraggoal, True, 'sim_seq3', 5)


#selection ratio plot
#-------------------------------------
#scene selection ratio vs selected ratio from baseline 
#------------------------------------
def plot_single_selection_ratio_by_inclusion_bins(df_full, df_selected, num_bins=10):
    """
    Plots the selection ratio per target_equal_ratio bin using Plotly.

    Parameters:
        df_full (pd.DataFrame): Full dataframe containing 'target_equal_ratio'.
        df_selected (pd.DataFrame): Filtered dataframe, subset of df_full.
        num_bins (int): Number of bins to divide the target_equal_ratio into (default: 10).
    """
    assert 'target_equal_ratio' in df_full.columns and 'target_equal_ratio' in df_selected.columns, \
        "Both dataframes must have an 'target_equal_ratio' column."

    # Define bin edges
    bins = np.linspace(0, 1.0, num_bins + 1)
    bin_labels = [f"{round(bins[i], 2)}–{round(bins[i+1], 2)}" for i in range(len(bins)-1)]

    # Assign bins
    df_full = df_full.copy()
    df_selected = df_selected.copy()
    df_full['bin'] = pd.cut(df_full['target_equal_ratio'], bins=bins, labels=bin_labels, include_lowest=True)
    df_selected['bin'] = pd.cut(df_selected['target_equal_ratio'], bins=bins, labels=bin_labels, include_lowest=True)

    # Count entries in each bin
    total_counts = df_full['bin'].value_counts().sort_index()
    selected_counts = df_selected['bin'].value_counts().sort_index()

    # Compute selection ratio
    selection_ratio = (selected_counts / total_counts).fillna(0)

    # Plot with Plotly
    fig = go.Figure()
    fig.add_trace(go.Bar(
        x=selection_ratio.index.astype(str),
        y=selection_ratio.values,
        marker=dict(color='steelblue'),
        name='Selection Ratio'
    ))

    fig.update_layout(
        title="Selection Ratio vs. Inclusion Ratio",
        xaxis_title="Inclusion Ratio (Binned)",
        yaxis_title="Selection Ratio",
        yaxis=dict(range=[0, 1]),
        width=900,
        height=600,
        bargap=0.1,
    )

    fig.show()

# draw without barplot
def plot_multiple_selection_vs_target_equal_ratio(data_pairs, bins=10, title_str=""):
    """
    Plots multiple selection vs. inclusion ratio lines in one graph.
    Args:
        data_pairs (list of tuples): Each tuple contains (df_total, df_selected, label).
        bins (int): Number of bins for target_equal_ratio.
    """

    # Create bins and labels
    bin_edges = np.linspace(0, 1, bins + 1)
    bin_centers = [(bin_edges[i] + bin_edges[i + 1]) / 2 for i in range(bins)]

    # make a list of linecolor, markertype, and linetype
    colors = ['blue', 'red', 'green', 'orange', 'purple', 'brown', 'teal', 'pink']
    dash_styles = ['solid', 'dot', 'dash', 'longdash', 'dashdot', 'longdashdot']
    marker_symbols = ['circle', 'square', 'diamond', 'cross', 'x', 'triangle-up']

    fig = go.Figure()

    for i, (df_total, df_selected, label) in enumerate(data_pairs):
        # Bin the inclusion ratios
        total_binned = pd.cut(df_total["target_equal_ratio"], bins=bin_edges, include_lowest=True)
        selected_binned = pd.cut(df_selected["target_equal_ratio"], bins=bin_edges, include_lowest=True)

        # Count per bin
        total_counts = total_binned.value_counts(sort=False)
        selected_counts = selected_binned.value_counts(sort=False)
        # print(f"bins num at {i} {selected_counts}")

        # Compute selection ratio
        selection_ratio = (selected_counts / total_counts).fillna(0)

        # Add line to plot
        fig.add_trace(go.Scatter(
            x=bin_centers,
            y=selection_ratio.values,
            mode='lines+markers',
            name=label,
            line=dict(color=colors[i % len(colors)], dash=dash_styles[i % len(dash_styles)]),
            marker=dict(symbol=marker_symbols[i % len(marker_symbols)], size=8),            
        ))

    # Layout settings
    fig.update_layout(
        title=dict(
        text=f"Selection Ratio vs. Inclusion Ratio ({title_str})",
        x=0.5,  # Center the title
        xanchor='center'  # Anchor the title at the center
        ),
        # title=f"Selection Ratio vs. Inclusion Ratio ({title_str})",
        xaxis_title="Inclusion Ratio",
        yaxis_title="Selection Ratio",
        xaxis=dict(range=[0, 1]),
        yaxis=dict(range=[0, 1]),
        width=500,
        height=500,
        legend=dict(title="Legend", x=0.01, y=0.99),
    )

    fig.show()


# PLOT
#-------------------------------------
# 베이스라인이 similarity의 변화에 따라서 어느 정도의 수용성 그래프를 보이는지 그린다.(false/positive + positive/positive)
# input (baselines, filtered_baselines, titlename)
# x = target_equal_ratio
# y = acceptance_rate per bin
#-------------------------------------

# plot_single_selection_ratio_by_inclusion_bins(df_eval_rag, efil3_both_rag, 8)
# plot_single_selection_ratio_by_inclusion_bins(df_eval_norag, efil3_both_norag, 8)
# plot_single_selection_ratio_by_inclusion_bins(df_eval_raggoal, efil3_raggoal, 8)
# plot_single_selection_ratio_by_inclusion_bins(df_eval_noraggoal, efil3_noraggoal, 8)


# plot_multiple_selection_vs_target_equal_ratio([
#     # (df_eval_rag, efil1_both_rag, "ours-(simple examiner)"),
#     # (df_eval_norag, efil1_both_norag, "norag-(simple examiner)"),
#     # (df_eval_rag, efil2_both_rag, "ours-(entity check examiner)"),
#     # (df_eval_norag, efil3_both_norag, "norag-(entity check examiner)"),
#     (df_eval_rag, efil3_seq_rag, "ours-(strict seq examiner)"),
#     (df_eval_norag, efil3_seq_norag, "norag-(strict seq examiner)"),    
# ], bins=8, title_str='stric sequence binary examiner')

# plot_multiple_selection_vs_target_equal_ratio([
#     (df_eval_rag, efil1_both_rag, "ours"),
#     (df_eval_norag, efil1_both_norag, "base1 (no RAG)"),
# ], bins=8, title_str='simple evaluator')

# plot_multiple_selection_vs_target_equal_ratio([
#     (df_eval_rag, efil2_both_rag, "ours"),
#     (df_eval_norag, efil2_both_norag, "base1 (no RAG)"),
# ], bins=8, title_str='entity evaluator')

# plot_multiple_selection_vs_target_equal_ratio([
#     (df_eval_rag, efil3_both_rag, "ours"),
#     (df_eval_norag, efil3_both_norag, "base1 (no RAG)"),
# ], bins=8, title_str='strict evaluator')

# plot_multiple_selection_vs_target_equal_ratio([
#     (df_eval_raggoal, efil3_raggoal, "base2 (no Taxonomy)"),
#     (df_eval_noraggoal, efil3_noraggoal, "base3 (no Taxonomy, no RAG)"),
# ], bins=8, title_str='strict evaluator')




In [ ]:
#------------------------------------------------
# Q0: Selection Ratios
#------------------------------------------------


# THIS IS FUNCTION WITHOUT ERROR BAR
# plot_multiple_selection_vs_target_equal_ratio([
#     (df_eval_rag, efil3_human_rag2, "ours-humaneval"),
#     (df_eval_rag, efil3_both_rag2, "ours-strict"),
#     (df_eval_norag, efil3_human_norag2, "norag-humaneval"),
#     (df_eval_norag, efil3_both_norag2, "norag-strict"),
# ], bins=5, title_str='strict evaluator')

# plot_multiple_selection_vs_target_equal_ratio([
#     (df_eval_rag, efil3_human_rag, "ours-humaneval"),
#     (df_eval_rag, efil3_both_rag, "ours-strict"),
#     (df_eval_norag, efil3_human_norag, "norag-humaneval"),
#     (df_eval_norag, efil3_both_norag, "norag-strict"),
# ], bins=5, title_str='strict evaluator')

# plot_multiple_selection_vs_target_equal_ratio([
#     (df_eval_rag, efil3_human_rag, "ours-human"),
#     (df_eval_norag, efil3_both_rag, "ours-strict"),
# ], bins=5, title_str='strict evaluator')

# plot_multiple_selection_vs_target_equal_ratio([
#     (df_eval_rag, efil3_human_norag, "baseline1-human"),
#     (df_eval_norag, efil3_both_norag, "baseline1-strict"),
# ], bins=5, title_str='strict evaluator')

# plot_multiple_selection_vs_target_equal_ratio([
#     (df_eval_raggoal, efil3_human_raggoal, "baseline2-human"),
#     (df_eval_raggoal, efil3_raggoal, "baseline2-strict"),
# ], bins=5, title_str='strict evaluator')

# plot_multiple_selection_vs_target_equal_ratio([
#     (df_eval_noraggoal, efil3_human_noraggoal, "baseline3-human"),
#     (df_eval_noraggoal, efil3_noraggoal, "baseline3-strict"),
# ], bins=5, title_str='strict evaluator')

# draw with barplot
def plot_similarity_vs_inclusion(data_pairs, bins=10, title_str="", column_name=""):
    """
    Plots similarity (mean ± std) vs. inclusion ratio for multiple (df_selected, label) pairs.
    
    Args:
        data_pairs (list of tuples): Each tuple contains (df_selected, label). df_selected must contain 'target_equal_ratio' and 'similarity'.
        bins (int): Number of bins for target_equal_ratio.
        title_str (str): Title for the plot.
    """
    bin_edges = np.linspace(0, 1, bins + 1)
    bin_centers = [(bin_edges[i] + bin_edges[i + 1]) / 2 for i in range(bins)]

    colors = ['blue', 'red', 'green', 'orange', 'purple', 'brown', 'teal', 'pink']
    dash_styles = ['solid', 'dot', 'dash', 'longdash', 'dashdot', 'longdashdot']
    marker_symbols = ['circle', 'square', 'diamond', 'cross', 'x', 'triangle-up']

    fig = go.Figure()

    for i, (df_selected, label) in enumerate(data_pairs):
        # Bin data
        selected_binned = pd.cut(df_selected["target_equal_ratio"], bins=bin_edges, include_lowest=True)
        mean_similarity = df_selected.groupby(selected_binned, observed=True)[column_name].mean()
        std_similarity = df_selected.groupby(selected_binned, observed=True)[column_name].std().fillna(0)

        # Count how many samples are in each bin
        counts = df_selected.groupby(selected_binned, observed=True).size()
        print(f"Counts per bin for {label}:")
        print(counts)


        # Add mean line with error bars (sticks)
        fig.add_trace(go.Scatter(
            x=bin_centers,
            y=mean_similarity.values,
            mode='lines+markers',
            name=label,
            line=dict(color=colors[i % len(colors)], dash=dash_styles[i % len(dash_styles)]),
            marker=dict(symbol=marker_symbols[i % len(marker_symbols)], size=8),
            error_y=dict(
                type='data',
                array=std_similarity.values,
                visible=True,
                color=colors[i % len(colors)]
            )
        ))

    fig.update_layout(
        title=dict(
        text=f"{title_str}",
        x=0.5,  # Center the title
        xanchor='center'  # Anchor the title at the center
        ),
        xaxis_title="Inclusion Ratio",
        yaxis_title="Similarity (mean ± std)",
        xaxis=dict(range=[0.0, 1]),
        yaxis=dict(range=[0, 1]),
        width=600,
        height=500,
        legend=dict(title="Legend", x=0.01, y=0.05),
    )

    fig.show()



def plot_similarity_vs_inclusion_trimmed(data_pairs, bins=5, title_str="", column_name=""):
    import numpy as np
    import pandas as pd
    import plotly.graph_objects as go

    bin_edges = np.linspace(0.0, 1.0, bins + 1)
    bin_centers = np.array([(bin_edges[i] + bin_edges[i + 1]) / 2 for i in range(bins)])

    colors = ['blue', 'red', 'green', 'orange', 'black']
    dash_styles = ['solid', 'dot', 'dash', 'longdash', 'dashdot']
    marker_symbols = ['circle', 'square', 'diamond', 'cross', 'triangle-up']

    fig = go.Figure()

    offset_step = 0.015  # horizontal offset amount
    for i, (df_selected, label) in enumerate(data_pairs):
        selected_binned = pd.cut(df_selected["target_equal_ratio"], bins=bin_edges, include_lowest=True)
        mean_similarity = df_selected.groupby(selected_binned, observed=True)[column_name].mean()
        std_similarity = df_selected.groupby(selected_binned, observed=True)[column_name].std().fillna(0)

        # Apply x-offset for side-by-side plotting
        offset = (i - len(data_pairs) / 2) * offset_step
        offset_bin_centers = bin_centers + offset

        fig.add_trace(go.Scatter(
            x=offset_bin_centers,
            y=mean_similarity.values,
            mode='lines+markers',
            name=label,
            line=dict(color=colors[i % len(colors)], dash=dash_styles[i % len(dash_styles)]),
            marker=dict(symbol=marker_symbols[i % len(marker_symbols)], size=8),
            error_y=dict(
                type='data',
                array=std_similarity.values,
                visible=True,
                color=colors[i % len(colors)],
                thickness=2.5  # Make error bars fatter
            )
        ))

    fig.update_layout(
        title=dict(
            text=f"{title_str}",
            x=0.5,
            xanchor='center'
        ),
        xaxis_title="Inclusion Ratio",
        yaxis_title="Similarity (mean ± std)",
        xaxis=dict(range=[0, 1.0]),
        yaxis=dict(range=[0, 1]),
        width=700,
        height=500,
        legend=dict(title="Baselines", x=0.65, y=0.05),
    )

    fig.show()

def compute_bin_stats(df, column_name, start=0.0, stop=1.0, bins=5):
    # Step 1: Define bin edges
    bin_edges = np.linspace(start, stop, bins + 1)
    df['bin'] = pd.cut(df['target_equal_ratio'], bins=bin_edges, include_lowest=True)

    # Step 2: Compute per-bin stats
    bin_stats = df.groupby('bin', observed=True)[column_name].agg(['mean', 'std', 'count']).reset_index()

    # Step 3: Overall stats
    overall_mean = df[column_name].mean()
    overall_std = df[column_name].std()
    overall_count = len(df)

    print("\n🔹 Per-bin stats:")
    print(bin_stats)

    print(f"\n🔸 Overall stats:\nMean = {overall_mean:.3f}, Std = {overall_std:.3f}, Count = {overall_count}")
    
    return bin_stats, overall_mean, overall_std

print("=================================================")
print("SIMILARITY vs INCLUSION")
print("=================================================")
#------------------------------------------------
# Q1: Similarity for EACH BASELINES under Human PASS/FAIL
#------------------------------------------------
# PLOT Similarities of SELECTED VALUES in each bin
# Remove first-bin data (e.g., 0–0.2 if 5 bins)
filtered_data_pairs = []
for df, label in [
    (yield_0, "direct"),
    (yield_1_1, "goal-norag"),
    (yield_1_2, "goal-rag"),
    (yield_2_1, "property-norag"),
    (yield_2_2, "property-rag (ours)"),
]:
    # df_filtered = df[df['target_equal_ratio'] > 0.2]  # adjust based on actual bin edges
    # filtered_data_pairs.append((df_filtered, label))
    filtered_data_pairs.append((df, label))

plot_similarity_vs_inclusion_trimmed(
    filtered_data_pairs,
    title_str="Inclusion Ratio vs Similarity between action sequences",
    bins=5,
    column_name='dtw_similarity'
)



filtered_data_pairs = []
for df, label in [
    (yield_2_1, "sps-norag"),
    (yield_2_2, "sps-rag"),
]:
    # df_filtered = df[df['target_equal_ratio'] > 0.2]  # adjust based on actual bin edges
    # filtered_data_pairs.append((df_filtered, label))
    filtered_data_pairs.append((df, label))

plot_similarity_vs_inclusion_trimmed(
    filtered_data_pairs,
    title_str="Inclusion Ratio vs Taxonomy Weighted Similarity",
    bins=5,
    column_name='dtw_similarity'
)





SIMILARITY vs INCLUSION


In [23]:
# TRUE POSITIVE / FALSE POSITIVE
def plot_confusion_matrix_bins4(df_true, df_false, df_positive, df_negative, classstr='class', N=5):
    import pandas as pd
    import numpy as np
    import plotly.graph_objects as go

    # Make clean copies
    df_true = df_true.copy()
    df_false = df_false.copy()
    df_positive = df_positive.copy()
    df_negative = df_negative.copy()

    # Define bin edges
    bin_edges = np.linspace(0, 1, N + 1)

    # Intersections to compute TP, FP, FN, TN correctly
    df_tp = pd.merge(df_true, df_positive, how='inner')
    df_fn = pd.merge(df_false, df_positive, how='inner')
    df_fp = pd.merge(df_false, df_negative, how='inner')
    df_tn = pd.merge(df_true, df_negative, how='inner')
    print(f"final number: {len(df_tp) + len(df_fn) + len(df_fp) + len(df_tn)}") 
    print(f"false negative {len(df_fn)}")
    # Assign bins
    for df in [df_tp, df_fp, df_fn, df_tn]:
        df['bin'] = pd.cut(df[classstr], bins=bin_edges, include_lowest=True)

    # Count values per bin
    tp_counts = df_tp['bin'].value_counts().sort_index()
    fp_counts = df_fp['bin'].value_counts().sort_index()
    fn_counts = df_fn['bin'].value_counts().sort_index()
    tn_counts = df_tn['bin'].value_counts().sort_index()

    # Align bins
    all_bins = sorted(set(tp_counts.index).union(fn_counts.index, fp_counts.index, tn_counts.index))
    bin_labels = [f"{round(interval.left, 2)}–{round(interval.right, 2)}" for interval in all_bins]

    # Create counts dataframe
    counts_df = pd.DataFrame(index=bin_labels)
    counts_df['TP'] = [tp_counts.get(b, 0) for b in all_bins]
    counts_df['FN'] = [fn_counts.get(b, 0) for b in all_bins]
    counts_df['FP'] = [fp_counts.get(b, 0) for b in all_bins]
    counts_df['TN'] = [tn_counts.get(b, 0) for b in all_bins]

    # Compute percentages
    counts_df['Total'] = counts_df.sum(axis=1)
    for col in ['TP', 'FN', 'FP', 'TN']:
        counts_df[f'{col}_pct'] = (
            (counts_df[col] / counts_df['Total'] * 100).round(1).astype(str) + '%'
        )

    # Plot
    fig = go.Figure()
    for label, color in zip(['TP', 'TN', 'FP', 'FN'], ['green', 'gray', 'red', 'black']):
        fig.add_trace(go.Bar(
            x=counts_df.index,
            y=counts_df[label],
            name=label,
            marker_color=color,
            text=counts_df[f'{label}_pct'],
            textposition='inside',
            insidetextanchor='middle',
            textfont=dict(color='white', size=12),
        ))

    fig.update_layout(
        title=f"Stacked Confusion Counts by Binned '{classstr}'",
        xaxis_title=f"{classstr} Bins",
        yaxis_title="Count",
        barmode='stack',
        width=950,
        height=500
    )

    fig.show()   

# score for all data
def plot_confusion_matrix_bins2(df_true, df_false, df_positive, df_negative, classstr='class', N=5):
    import pandas as pd
    import numpy as np
    import plotly.graph_objects as go

    # Make clean copies
    df_true = df_true.copy()
    df_false = df_false.copy()
    df_positive = df_positive.copy()
    df_negative = df_negative.copy()

    # Define bin edges
    bin_edges = np.linspace(0, 1, N + 1)

    # Intersections to compute TP, FP, FN, TN correctly
    df_tp = pd.merge(df_true, df_positive, how='inner')
    df_fn = pd.merge(df_false, df_positive, how='inner')
    df_fp = pd.merge(df_false, df_negative, how='inner')
    df_tn = pd.merge(df_true, df_negative, how='inner')

    # Print final counts
    total = len(df_tp) + len(df_fn) + len(df_fp) + len(df_tn)
    print(f"Final total: {total}")
    print(f"TP: {len(df_tp)} | TN: {len(df_tn)} | FP: {len(df_fp)} | FN: {len(df_fn)}")

    # Assign bins
    for df in [df_tp, df_fp, df_fn, df_tn]:
        df['bin'] = pd.cut(df[classstr], bins=bin_edges, include_lowest=True)

    # Count values per bin
    tp_counts = df_tp['bin'].value_counts().sort_index()
    fp_counts = df_fp['bin'].value_counts().sort_index()
    fn_counts = df_fn['bin'].value_counts().sort_index()
    tn_counts = df_tn['bin'].value_counts().sort_index()

    # Align bins
    all_bins = sorted(set(tp_counts.index).union(fn_counts.index, fp_counts.index, tn_counts.index))
    bin_labels = [f"{round(interval.left, 2)}–{round(interval.right, 2)}" for interval in all_bins]

    # Create counts dataframe
    counts_df = pd.DataFrame(index=bin_labels)
    counts_df['TP'] = [tp_counts.get(b, 0) for b in all_bins]
    counts_df['FN'] = [fn_counts.get(b, 0) for b in all_bins]
    counts_df['FP'] = [fp_counts.get(b, 0) for b in all_bins]
    counts_df['TN'] = [tn_counts.get(b, 0) for b in all_bins]

    # Compute percentages
    counts_df['Total'] = counts_df.sum(axis=1)
    for col in ['TP', 'FN', 'FP', 'TN']:
        counts_df[f'{col}_pct'] = (
            (counts_df[col] / counts_df['Total'] * 100).round(1).astype(str) + '%'
        )

    # Plot
    fig = go.Figure()
    for label, color in zip(['TP', 'TN', 'FP', 'FN'], ['green', 'gray', 'red', 'black']):
        fig.add_trace(go.Bar(
            x=counts_df.index,
            y=counts_df[label],
            name=label,
            marker_color=color,
            text=counts_df[f'{label}_pct'],
            textposition='inside',
            insidetextanchor='middle',
            textfont=dict(color='white', size=12),
        ))

    fig.update_layout(
        title=f"Stacked Confusion Counts by Binned '{classstr}'",
        xaxis_title=f"{classstr} Bins",
        yaxis_title="Count",
        barmode='stack',
        width=950,
        height=500
    )

    fig.show()

    # === Compute Global Metrics ===
    TP, TN, FP, FN = len(df_tp), len(df_tn), len(df_fp), len(df_fn)
    accuracy = (TP + TN) / (TP + TN + FP + FN) if (TP + TN + FP + FN) > 0 else 0
    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    print("\n=== Global Metrics ===")
    print(f"Accuracy:  {accuracy:.3f}")
    print(f"Precision: {precision:.3f}")
    print(f"Recall:    {recall:.3f}")
    print(f"F1 Score:  {f1:.3f}")

# scores per bin
def plot_confusion_matrix_bins3_old(df_true, df_false, df_positive, df_negative, classstr='class', N=5):
    import pandas as pd
    import numpy as np
    import plotly.graph_objects as go

    # Make clean copies
    df_true = df_true.copy()
    df_false = df_false.copy()
    df_positive = df_positive.copy()
    df_negative = df_negative.copy()

    # Define bin edges
    bin_edges = np.linspace(0, 1, N + 1)

    # Intersections to compute TP, FP, FN, TN correctly
    df_tp = pd.merge(df_true, df_positive, how='inner')
    df_fn = pd.merge(df_false, df_positive, how='inner')
    df_fp = pd.merge(df_false, df_negative, how='inner')
    df_tn = pd.merge(df_true, df_negative, how='inner')

    # Print final counts
    total = len(df_tp) + len(df_fn) + len(df_fp) + len(df_tn)
    print(f"Final total: {total}")
    print(f"TP: {len(df_tp)} | TN: {len(df_tn)} | FP: {len(df_fp)} | FN: {len(df_fn)}")

    # Assign bins
    for df in [df_tp, df_fp, df_fn, df_tn]:
        df['bin'] = pd.cut(df[classstr], bins=bin_edges, include_lowest=True)

    # Count values per bin
    tp_counts = df_tp['bin'].value_counts().sort_index()
    fp_counts = df_fp['bin'].value_counts().sort_index()
    fn_counts = df_fn['bin'].value_counts().sort_index()
    tn_counts = df_tn['bin'].value_counts().sort_index()

    # Align bins
    all_bins = sorted(set(tp_counts.index).union(fn_counts.index, fp_counts.index, tn_counts.index))
    bin_labels = [f"{round(interval.left, 2)}–{round(interval.right, 2)}" for interval in all_bins]

    # Create counts dataframe
    counts_df = pd.DataFrame(index=bin_labels)
    counts_df['TP'] = [tp_counts.get(b, 0) for b in all_bins]
    counts_df['FN'] = [fn_counts.get(b, 0) for b in all_bins]
    counts_df['FP'] = [fp_counts.get(b, 0) for b in all_bins]
    counts_df['TN'] = [tn_counts.get(b, 0) for b in all_bins]

    # Compute percentages
    counts_df['Total'] = counts_df[['TP', 'FN', 'FP', 'TN']].sum(axis=1)
    for col in ['TP', 'FN', 'FP', 'TN']:
        counts_df[f'{col}_pct'] = (
            (counts_df[col] / counts_df['Total'] * 100).round(1).astype(str) + '%'
        )

    # Compute per-bin metrics
    def safe_div(n, d):
        return n / d if d != 0 else 0

    counts_df['accuracy'] = counts_df.apply(lambda row: safe_div(row['TP'] + row['TN'], row['Total']), axis=1)
    counts_df['precision'] = counts_df.apply(lambda row: safe_div(row['TP'], row['TP'] + row['FP']), axis=1)
    counts_df['recall'] = counts_df.apply(lambda row: safe_div(row['TP'], row['TP'] + row['FN']), axis=1)
    counts_df['f1'] = counts_df.apply(lambda row: safe_div(2 * row['precision'] * row['recall'],
                                                           row['precision'] + row['recall']), axis=1)

    # Plot
    fig = go.Figure()
    for label, color in zip(['TP', 'TN', 'FP', 'FN'], ['green', 'gray', 'red', 'black']):
        fig.add_trace(go.Bar(
            x=counts_df.index,
            y=counts_df[label],
            name=label,
            marker_color=color,
            text=counts_df[f'{label}_pct'],
            textposition='inside',
            insidetextanchor='middle',
            textfont=dict(color='white', size=12),
        ))

    fig.update_layout(
        title=dict(
        text=f"Stacked Confusion Counts by Binned '{classstr}'",
        x=0.5,  # Center the title
        xanchor='center'  # Anchor the title at the center
        ),        
        xaxis_title=f"{classstr} Bins",
        yaxis_title="Count",
        barmode='stack',
        width=950,
        height=500
    )
    fig.show()

    # Print per-bin metrics
    print("\n=== Per-bin Metrics ===")
    for idx, row in counts_df.iterrows():
        print(f"\nBin {idx}:")
        print(f"  Accuracy:  {row['accuracy']:.3f}")
        print(f"  Precision: {row['precision']:.3f}")
        print(f"  Recall:    {row['recall']:.3f}")
        print(f"  F1 Score:  {row['f1']:.3f}")


def plot_confusion_matrix_bins3(df_true, df_false, df_positive, df_negative, classstr='class', N=5, title_str=None):
    import pandas as pd
    import numpy as np
    import plotly.graph_objects as go

    # Make clean copies
    df_true = df_true.copy()
    df_false = df_false.copy()
    df_positive = df_positive.copy()
    df_negative = df_negative.copy()

    # Define bin edges
    bin_edges = np.linspace(0, 1, N + 1)

    # Intersections to compute TP, FP, FN, TN correctly
    df_tp = pd.merge(df_true, df_positive, how='inner')
    df_fn = pd.merge(df_false, df_positive, how='inner')
    df_fp = pd.merge(df_false, df_negative, how='inner')
    df_tn = pd.merge(df_true, df_negative, how='inner')

    # Print final counts
    total = len(df_tp) + len(df_fn) + len(df_fp) + len(df_tn)
    print(f"Final total: {total}")
    print(f"TP: {len(df_tp)} | TN: {len(df_tn)} | FP: {len(df_fp)} | FN: {len(df_fn)}")

    # Assign bins
    for df in [df_tp, df_fp, df_fn, df_tn]:
        df['bin'] = pd.cut(df[classstr], bins=bin_edges, include_lowest=True)

    # Count values per bin
    tp_counts = df_tp['bin'].value_counts().sort_index()
    fp_counts = df_fp['bin'].value_counts().sort_index()
    fn_counts = df_fn['bin'].value_counts().sort_index()
    tn_counts = df_tn['bin'].value_counts().sort_index()

    # Align bins
    all_bins = sorted(set(tp_counts.index).union(fn_counts.index, fp_counts.index, tn_counts.index))
    bin_labels = [f"{round(interval.left, 2)}–{round(interval.right, 2)}" for interval in all_bins]

    # Create counts dataframe
    counts_df = pd.DataFrame(index=bin_labels)
    counts_df['TP'] = [tp_counts.get(b, 0) for b in all_bins]
    counts_df['FN'] = [fn_counts.get(b, 0) for b in all_bins]
    counts_df['FP'] = [fp_counts.get(b, 0) for b in all_bins]
    counts_df['TN'] = [tn_counts.get(b, 0) for b in all_bins]

    # Compute percentages
    counts_df['Total'] = counts_df[['TP', 'FN', 'FP', 'TN']].sum(axis=1)
    for col in ['TP', 'FN', 'FP', 'TN']:
        counts_df[f'{col}_pct'] = (
            (counts_df[col] / counts_df['Total'] * 100).round(1).astype(str) + '%'
        )

    # Compute per-bin metrics
    def safe_div(n, d):
        return n / d if d != 0 else 0

    counts_df['accuracy'] = counts_df.apply(lambda row: safe_div(row['TP'] + row['TN'], row['Total']), axis=1)
    counts_df['precision'] = counts_df.apply(lambda row: safe_div(row['TP'], row['TP'] + row['FP']), axis=1)
    counts_df['recall'] = counts_df.apply(lambda row: safe_div(row['TP'], row['TP'] + row['FN']), axis=1)
    counts_df['f1'] = counts_df.apply(lambda row: safe_div(2 * row['precision'] * row['recall'],
                                                           row['precision'] + row['recall']), axis=1)

    # Default title
    if title_str is None:
        title_str = f"Stacked Confusion Counts by Binned '{classstr}'"

    # Plot
    fig = go.Figure()
    for label, color in zip(['TP', 'TN', 'FP', 'FN'], ['green', 'gray', 'red', 'black']):
        fig.add_trace(go.Bar(
            x=counts_df.index,
            y=counts_df[label],
            name=label,
            marker_color=color,
            text=counts_df[f'{label}_pct'],
            textposition='inside',
            insidetextanchor='middle',
            textfont=dict(color='white', size=12),
        ))

    fig.update_layout(
        title=dict(
            text=title_str,
            x=0.5,
            xanchor='center'
        ),
        xaxis_title=f"{classstr} Bins",
        yaxis_title="Count",
        barmode='stack',
        bargap=0.4,  # Make bars slimmer
        width=600,
        height=500
    )

    fig.show()

    # Print per-bin metrics
    print("\n=== Per-bin Metrics ===")
    for idx, row in counts_df.iterrows():
        print(f"\nBin {idx}:")
        print(f"  Accuracy:  {row['accuracy']:.3f}")
        print(f"  Precision: {row['precision']:.3f}")
        print(f"  Recall:    {row['recall']:.3f}")
        print(f"  F1 Score:  {row['f1']:.3f}")


def plot_binary_stack_by_bin_old(df_a, df_b, classstr='class', N=5, label_a='A', label_b='B'):
    # Define bin edges
    bin_edges = np.linspace(0, 1, N + 1)

    # Make copies to avoid warnings
    df_a = df_a.copy()
    df_b = df_b.copy()

    # Assign bins
    df_a.loc[:, 'bin'] = pd.cut(df_a[classstr], bins=bin_edges, include_lowest=True)
    df_b.loc[:, 'bin'] = pd.cut(df_b[classstr], bins=bin_edges, include_lowest=True)

    # Count values in each bin
    a_counts = df_a['bin'].value_counts().sort_index()
    b_counts = df_b['bin'].value_counts().sort_index()

    # Ensure all bins present
    all_bins = sorted(set(a_counts.index).union(b_counts.index))
    bin_labels = [f"{round(interval.left, 2)}–{round(interval.right, 2)}" for interval in all_bins]

    # Create counts DataFrame
    counts_df = pd.DataFrame(index=bin_labels)
    counts_df[label_a] = [a_counts.get(b, 0) for b in all_bins]
    counts_df[label_b] = [b_counts.get(b, 0) for b in all_bins]

    # Calculate total and percentage
    counts_df['Total'] = counts_df[label_a] + counts_df[label_b]
    for label in [label_a, label_b]:
        counts_df[f'{label}_pct'] = (counts_df[label] / counts_df['Total'] * 100).round(1).astype(str) + '%'

    # Plot
    fig = go.Figure()
    for label, color in zip([label_a, label_b], ['green', 'red']):
        fig.add_trace(go.Bar(
            x=counts_df.index,
            y=counts_df[label],
            name=label,
            marker_color=color,
            text=counts_df[f'{label}_pct'],
            textposition='inside',
            insidetextanchor='middle',
            textfont=dict(color='white', size=12),
        ))

    fig.update_layout(
        title=dict(
        text=f"Stacked Bar of '{label_a}' and '{label_b}' by binned '{classstr}'",
        x=0.5,  # Center the title
        xanchor='center'  # Anchor the title at the center
        ),
        xaxis_title=f"{classstr} Bins",
        yaxis_title="Count",
        barmode='stack',
        width=800,
        height=500
    )

    fig.show() 

def plot_binary_stack_by_bin(df_a, df_b, classstr='class', N=5, label_a='A', label_b='B', title_str=None):
    import numpy as np
    import pandas as pd
    import plotly.graph_objects as go

    # Define bin edges
    bin_edges = np.linspace(0, 1, N + 1)

    # Make copies to avoid SettingWithCopyWarning
    df_a = df_a.copy()
    df_b = df_b.copy()

    # Assign bins
    df_a['bin'] = pd.cut(df_a[classstr], bins=bin_edges, include_lowest=True)
    df_b['bin'] = pd.cut(df_b[classstr], bins=bin_edges, include_lowest=True)

    # Count values in each bin
    a_counts = df_a['bin'].value_counts().sort_index()
    b_counts = df_b['bin'].value_counts().sort_index()

    # Ensure consistent bin labels
    all_bins = sorted(set(a_counts.index).union(b_counts.index))
    bin_labels = [f"{round(interval.left, 2)}–{round(interval.right, 2)}" for interval in all_bins]

    # Create counts DataFrame
    counts_df = pd.DataFrame(index=bin_labels)
    counts_df[label_a] = [a_counts.get(b, 0) for b in all_bins]
    counts_df[label_b] = [b_counts.get(b, 0) for b in all_bins]

    # Calculate total and percentage
    counts_df['Total'] = counts_df[label_a] + counts_df[label_b]
    for label in [label_a, label_b]:
        counts_df[f'{label}_pct'] = (
            (counts_df[label] / counts_df['Total'] * 100).round(1).astype(str) + '%'
        )

    # Plot
    fig = go.Figure()
    for label, color in zip([label_a, label_b], ['green', 'red']):
        fig.add_trace(go.Bar(
            x=counts_df.index,
            y=counts_df[label],
            name=label,
            marker_color=color,
            text=counts_df[f'{label}_pct'],
            textposition='inside',
            insidetextanchor='middle',
            textfont=dict(color='white', size=12),
        ))

    # Title logic
    if title_str is None:
        title_str = f"Stacked Bar of '{label_a}' and '{label_b}' by binned '{classstr}'"

    fig.update_layout(
        title=dict(
            text=title_str,
            x=0.5,
            xanchor='center'
        ),
        xaxis_title=f"{classstr} Bins",
        yaxis_title="Count",
        barmode='stack',
        bargap=0.25,  # Increase gap to make bars slimmer
        width=800,
        height=500
    )

    fig.show()

# Plot True Positive and False Positive in Bar Graph for all strict baselines
#Select한 ratio가 높은게 좋은게 아니라, 정답중에 얼마나 골랐느냐와 고른 것중에 정답이 얼마나 있었는가가 더 중요하다.

#=======================
# POSITIVES & NEGATIVES
#=======================
#strict rules
rag_positive = efil3_both_rag #POSITIVE
rag_negative = df_eval_rag.drop(efil3_both_rag.index) #NEGETIVE
rag_true = efil3_human_rag #TRUE
rag_false = df_eval_rag.drop(efil3_human_rag.index) #FALSE
print(rag_positive.head(20))
print(rag_negative.head(20))


norag_positive = efil3_both_norag
norag_negative = df_eval_norag.drop(efil3_both_norag.index)
norag_true = efil3_human_norag
norag_false = df_eval_norag.drop(efil3_human_norag.index)

#------lenient rules------ for (all entity, all sim, only sequence)
# rag_positive = efil3_both_rag2 #TP
# rag_negative = df_eval_rag.drop(efil3_both_rag2.index) #TN
# rag_true = efil3_human_rag2 #
# rag_false = df_eval_rag.drop(efil3_human_rag2.index)

# norag_positive = efil3_both_norag2
# norag_negative = df_eval_norag.drop(efil3_both_norag2.index)
# norag_true = efil3_human_norag2
# norag_false = df_eval_norag.drop(efil3_human_norag2.index)
#--------------------------

raggoal_positive = efil3_raggoal
raggoal_negative = df_eval_raggoal.drop(efil3_raggoal.index)
raggoal_true = efil3_human_raggoal
raggoal_false = df_eval_raggoal.drop(efil3_human_raggoal.index)

noraggoal_positive = efil3_noraggoal
noraggoal_negative = df_eval_noraggoal.drop(efil3_noraggoal.index)
noraggoal_true = efil3_human_noraggoal
noraggoal_false = df_eval_noraggoal.drop(efil3_human_noraggoal.index)


#=======================
# Drawing each values
#=======================
#N=1
print(rag_true.shape[0])
print(rag_false.shape[0])
print(rag_positive.shape[0])
print(rag_negative.shape[0])


plot_confusion_matrix_bins3(rag_true, rag_false,rag_positive, rag_negative, classstr='target_equal_ratio', N=1, title_str="Stacked Confusion Chart (taxo-med-rag)")
 
plot_confusion_matrix_bins3(norag_true, norag_false, norag_positive, norag_negative, classstr='target_equal_ratio', N=1, title_str="Stacked Confusion Chart (taxo-med)")

plot_confusion_matrix_bins3(raggoal_true, raggoal_false, raggoal_positive, raggoal_negative, classstr='target_equal_ratio', N=1, title_str="Stacked Confusion Chart (goal-med-rag)")

plot_confusion_matrix_bins3(noraggoal_true, noraggoal_false, noraggoal_positive, noraggoal_negative, classstr='target_equal_ratio', N=1, title_str="Stacked Confusion Chart (goal-med)")


plot_confusion_matrix_bins3(rag_true, rag_false,rag_positive, rag_negative, classstr='target_equal_ratio', N=3, title_str="Stacked Confusion Chart (taxo-med-rag)")
 
plot_confusion_matrix_bins3(norag_true, norag_false, norag_positive, norag_negative, classstr='target_equal_ratio', N=3, title_str="Stacked Confusion Chart (taxo-med)")

plot_confusion_matrix_bins3(raggoal_true, raggoal_false, raggoal_positive, raggoal_negative, classstr='target_equal_ratio', N=3, title_str="Stacked Confusion Chart (goal-med-rag)")

plot_confusion_matrix_bins3(noraggoal_true, noraggoal_false, noraggoal_positive, noraggoal_negative, classstr='target_equal_ratio', N=3, title_str="Stacked Confusion Chart (goal-med)")



# def plot_binary_stack_by_bin(df_a, df_b, classstr='class', N=5, label_a='A', label_b='B'):
plot_binary_stack_by_bin(rag_true, rag_false, classstr='target_equal_ratio', N=5, label_a='TRUE', label_b='FAlSE')

plot_binary_stack_by_bin(rag_positive, rag_negative, classstr='target_equal_ratio', N=5, label_a='POSITIVE', label_b='NEGATIVE')

plot_binary_stack_by_bin(norag_positive, norag_negative, classstr='target_equal_ratio', N=5, label_a='POSITIVE', label_b='NEGATIVE')

plot_binary_stack_by_bin(raggoal_positive, raggoal_negative, classstr='target_equal_ratio', N=5, label_a='POSITIVE', label_b='NEGATIVE')
#N=3
# plot_confusion_matrix_bins3(rag_true, rag_false,rag_positive, rag_negative, classstr='target_equal_ratio', N=3)

# plot_confusion_matrix_bins3(norag_true, norag_false, norag_positive, norag_negative, classstr='target_equal_ratio', N=3)

# plot_confusion_matrix_bins3(raggoal_true, raggoal_false, raggoal_positive, raggoal_negative, classstr='target_equal_ratio', N=3)

# plot_confusion_matrix_bins3(noraggoal_true, noraggoal_false, noraggoal_positive, noraggoal_negative, classstr='target_equal_ratio', N=3)

NameError: name 'efil3_both_rag' is not defined

In [3]:
# core activity and goal description 
import openai
openai.api_key = os.getenv("OPENAI_API_KEY")  

# def get_embedding(text):
#     response = client.embeddings.create(
#         model="text-embedding-3-large",
#         input=text
#     )
#     return response.data[0].embedding

def get_embedding(text):
    response = openai.embeddings.create(
        model="text-embedding-3-large",
        input=[text]
    )
    return response.data[0].embedding


source_idx_list = [i for i in range(213//3) for _ in range(3)]
print(source_idx_list)
print(len(source_idx_list))
goalsim1 = []
goalsim2 = []
goalsim3 = []
goalsim4 = []
print(len(df_eval_rag))
print(len(df_eval_norag))
print(len(df_eval_raggoal))
print(len(df_eval_noraggoal))



for i in range(len(source_idx_list)-1, len(source_idx_list)):
    # print(i)
    source_video_idx = source_idx_list[i]
    goalstep_test_video_list = database_init.goalstep_test_video_list
    source_goalstep_video = goalstep_test_video_list[source_video_idx]    

    print( f"source video {source_goalstep_video['video_uid']}" )
    # print( f"goal_desc {source_goalstep_video['goal_description']}" )
    # print( f"core_activity {df_eval_rag.iloc[i]['core_activity']}" )

    
    embedding_description = get_embedding(source_goalstep_video['goal_description'])
    embedding_rag = get_embedding(df_eval_rag.iloc[i]['core_activity'])
    embedding_norag = get_embedding(df_eval_norag.iloc[i]['core_activity'])
    embedding_raggoal = get_embedding(df_eval_raggoal.iloc[i]['core_activity'])
    # embedding_noraggoal = get_embedding(df_eval_noraggoal.iloc[i]['core_activity'])

    similarity1 = cosine_similarity(np.array(embedding_description), np.array(embedding_rag))
    similarity2 = cosine_similarity(np.array(embedding_description), np.array(embedding_norag))
    similarity3 = cosine_similarity(np.array(embedding_description), np.array(embedding_raggoal))
    # similarity4 = cosine_similarity(np.array(embedding_description), np.array(embedding_noraggoal))
    
    print(similarity1)
    print(similarity2)
    print(similarity3)
    goalsim1.append(similarity1)
    goalsim2.append(similarity2)
    goalsim3.append(similarity3)
    # goalsim4.append(similarity4)

# print(goalsim1)
# print(goalsim2)
# print(goalsim3)
# print(goalsim4)
# print(len(goalsim1))
# print(len(goalsim2))
# print(len(goalsim3))
# print(len(goalsim4))


NameError: name 'os' is not defined

In [73]:



# print(goalsim1)
# print(goalsim2)
# print(goalsim3)
# print(goalsim4)


print(df_eval_noraggoal.iloc[212]['core_activity'])


nan


In [76]:
similarity1=[0.5778636637447012, 0.582974897182386, 0.5778284959124516, 0.679542106957833, 0.6436508520286807, 0.679542106957833, 0.5483460386832888, 0.5483244494789303, 0.5483460386832888, 0.7705137145340148, 0.7114514054869984, 0.7704460845681002, 0.6325576995118218, 0.589508238521243, 0.6325802688691654, 0.7744213595688373, 0.7398820411007674, 0.5409185542065786, 0.6721210949151936, 0.6720836780494432, 0.6720195044491901, 0.6128996927323961, 0.6128996927323961, 0.9170288585563074, 0.38617919327804573, 0.38616453838135334, 0.38616453838135334, 0.6776448207915785, 0.6774544468701627, 0.6774373890014296, 0.6524793125515148, 0.6525233548081956, 0.670201359262916, 0.4307649610668214, 0.4307649610668214, 0.4307649610668214, 0.3341355650824397, 0.3449026844302219, 0.34481826961991596, 0.5408675552746434, 0.5408675552746434, 0.5408675552746434, 0.25735798506004315, 0.25735798506004315, 0.25740455906370646, 0.47624672582462374, 0.47624672582462374, 0.47624672582462374, 0.39301777440782615, 0.49765122250028365, 0.39301777440782615, 0.557090942972094, 0.5771974724641636, 0.557090942972094, 0.4485141151772602, 0.4485141151772602, 0.47506154443901544, 0.4334910297085468, 0.5417568548473599, 0.4334252189112176, 0.41317893804230205, 0.41317893804230205, 0.41317893804230205, 0.46969598025622333, 0.5449208303465614, 0.46969598025622333, 0.40548632929391404, 0.40548632929391404, 0.405457356015192, 0.3537378236832194, 0.327415362723898, 0.28218294778507513, 0.7990514287184614, 0.7990301388946973, 0.799024608484631, 0.5130483000828613, 0.5130483000828613, 0.5105820987928431, 0.7256648965939693, 0.7256076255241531, 0.7256648965939693, 0.7185290119506106, 0.7185290119506106, 0.7185290119506106, 0.694718896076403, 0.6953022099977036, 0.6946641376601989, 0.31455895726994637, 0.3145525152162654, 0.31455895726994637, 0.39875130558791905, 0.3716429506872173, 0.35314196562446726, 0.6285916440317936, 0.642838133007802, 0.6429188955606232, 0.47691026331058317, 0.47697190561478203, 0.47691026331058317, 0.6174370690258875, 0.46157274897942474, 0.5000071258394309, 0.48498613071790553, 0.46351202318495677, 0.46351202318495677, 0.58557187586642, 0.58557187586642, 0.6204193597294554, 0.5091187335232681, 0.5092528369782692, 0.5092528369782692, 0.7404215830750367, 0.8694092606356736, 0.8694092606356736, 0.24217799988440128, 0.23732106742165782, 0.2373130580573219, 0.8149869618463819, 0.7498228569221376, 0.7425164077638325, 0.8656840941652026, 0.8657080815203794, 0.8657106529747386, 0.36641605672488886, 0.36818186308539075, 0.36646057762948997, 0.5392093997243067, 0.5354907616252497, 0.5392093997243067, 0.3536987994290063, 0.3537155061089267, 0.3536987994290063, 0.5842487748233518, 0.5842487748233518, 0.5842487748233518, 0.39239496621408587, 0.392341865014541, 0.39232670906738404, 0.8384815217604682, 0.8384815217604682, 0.8384815217604682, 0.7819107090333783, 0.7819486540111621, 0.7819486540111621, 0.288233010676044, 0.2882415571862892, 0.3942077396363303, 0.3264425575590685, 0.3264425575590685, 0.326457788439521, 0.759013486352823, 0.759013486352823, 0.3888195020888615, 0.7022789744280139, 0.7022031191369117, 0.7022789744280139, 0.8880398760549424, 0.8880398760549424, 0.8880398760549424, 0.15617376813290915, 0.1561892127473618, 0.15621627919879916, 0.8286782017357481, 0.8287067495860981, 0.8287086633468697, 0.6902239526782921, 0.6902239526782921, 0.6902407850367202, 0.5657193507809647, 0.5657193507809647, 0.5657193507809647, 0.802242057720892, 0.8527690418801647, 0.802242057720892, 0.3840288234517097, 0.38402441601749604, 0.38405652780516053, 0.3163796451963766, 0.3162978737220522, 0.3163059454093794, 0.2071106597824005, 0.20704985895654948, 0.20704985895654948, 0.424802837304989, 0.4247564309672468, 0.4247564309672468, 0.2640132208018344, 0.2516608417257411, 0.3460022145615208, 0.48814310725734567, 0.48811644564481355, 0.48817151553652377, 0.37158770884013037, 0.37158770884013037, 0.37158770884013037, 0.4694627188763364, 0.46945173854990163, 0.46966188459974356, 0.297615841041411, 0.27882005118197617, 0.29768904663996737, 0.30569686977128474, 0.30569686977128474, 0.30569686977128474, 0.6143577430185075, 0.6143577430185075, 0.6159759291964962, 0.41502221316017135, 0.3858066630117833, 0.41502221316017135, 0.3635975122682438, 0.36351278362222283, 0.3635432287456949]

similarity2=[0.7996235604519438, 0.7505865952990389, 0.7996235604519438, 0.6436508520286807, 0.679542106957833, 0.6794971893555375, 0.7320037325458781, 0.7320037325458781, 0.7320037325458781, 0.7705137145340148, 0.7704460845681002, 0.7704460845681002, 0.6195611824309806, 0.6564771219018858, 0.6564771219018858, 0.599551425086088, 0.599551425086088, 0.599551425086088, 0.6795807740859883, 0.6795807740859883, 0.6795398046747823, 0.6128996927323961, 0.6128472696622601, 0.612620617315679, 0.4657714873033477, 0.20313262958748873, 0.39504171853899084, 0.7649170809251293, 0.764915047233493, 0.7649170809251293, 0.6524793125515148, 0.6524793125515148, 0.6525446777159358, 0.5525842145325672, 0.5525842145325672, 0.5525842145325672, 0.34481826961991596, 0.34493624171027965, 0.3341355650824397, 0.47463767200835677, 0.4173317130907538, 0.4173317130907538, 0.25735798506004315, 0.25735798506004315, 0.25740455906370646, 0.24114346770797848, 0.24114346770797848, 0.24114346770797848, 0.39301777440782615, 0.39301777440782615, 0.3929758471263424, 0.557090942972094, 0.5570454022578315, 0.5821714822114508, 0.38144611408149326, 0.38144611408149326, 0.4778488601035727, 0.4334399678317037, 0.4334910297085468, 0.3889500607898847, 0.41317893804230205, 0.41317893804230205, 0.41317893804230205, 0.3342417043450787, 0.32670115062055255, 0.2115945539073312, 0.6193084465542829, 0.5306282769504033, 0.5305833853058023, 0.3537378236832194, 0.28218294778507513, 0.3274435296222377, 0.9999990476852048, 1.0000000000000002, 1.0000000000000002, 0.6883347525012372, 0.6883347525012372, 0.6883901056645582, 0.7256648965939693, 0.7256648965939693, 0.7256648965939693, 0.7185290119506106, 0.7185290119506106, 0.7185019419849975, 0.8122172870606168, 0.722075916267506, 0.7668273862143209, 0.2667124786106884, 0.2560461838570885, 0.2667124786106884, 0.42829135875632385, 0.5143416302458235, 0.5143416302458235, 0.6410740422148599, 0.6410094370177746, 0.6086836907165005, 0.4766499775347174, 0.47678595257763273, 0.47691026331058317, 0.45187513955106356, 0.4018062534042619, 0.5000071258394309, 0.4213888619221459, 0.4213888619221459, 0.4852134484021161, 0.5503958158059123, 0.5913557728459251, 0.6113648383049102, 0.7736734667830533, 0.7737222874939718, 0.7736734667830533, 0.7404215830750367, 0.9244798511819445, 0.7650925456542989, 0.27224746109117215, 0.32798163716781703, 0.27224746109117215, 0.7607327962167714, 0.7348161895346612, 0.7348161895346612, 0.6500383353623543, 0.8657080815203794, 0.6500383353623543, 0.36818186308539075, 0.43328299888071914, 0.43328299888071914, 0.5392093997243067, 0.5354948361740165, 0.7783033052822609, 0.3258342116272503, 0.37977927517293525, 0.325877058550351, 0.3477622829198257, 0.3477622829198257, 0.3477622829198257, 0.576387278165039, 0.5762827681974464, 0.4909236111143983, 0.6549386609419069, 0.6549386609419069, 0.6504485691697728, 0.7819107090333783, 0.7819486540111621, 0.7819486540111621, 0.39426312404259073, 0.39416837651442194, 0.3942077396363303, 0.29999394555258685, 0.29999394555258685, 0.29999394555258685, 0.7357665174821139, 0.8062754777540277, 0.8062047200285555, 0.5941204018230147, 0.6856537158862475, 0.6700350676500797, 0.7356055439272154, 0.7356055439272154, 0.8880398760549424, 0.3422956525409453, 0.15617376813290915, 0.15621627919879916, 0.8730221860260554, 0.8287067495860981, 0.8730870722227939, 0.6402869377846522, 0.6402869377846522, 0.6402869377846522, 0.5657193507809647, 0.5657193507809647, 0.5907343930279394, 0.31813734142869177, 0.31813734142869177, 0.31813734142869177, 0.38402441601749604, 0.38402441601749604, 0.38405652780516053, 0.35820882180326863, 0.358128139481723, 0.3581322513426185, 0.5494294845609967, 0.5483328600379044, 0.5483328600379044, 0.4247564309672468, 0.4247564309672468, 0.4247564309672468, 0.3460022145615208, 0.3460022145615208, 0.3460022145615208, 0.48814310725734567, 0.4879851016346898, 0.48817151553652377, 0.3708752421102667, 0.3708752421102667, 0.3708752421102667, 0.33176599344548674, 0.3317678923614604, 0.331836535311467, 0.2975600471762408, 0.29760426389670497, 0.2976428498952826, 0.3875111405078753, 0.2134181406633088, 0.29705268133757046, 0.6143352668811315, 0.6143577430185075, 0.6159759291964962, 0.38572190201429807, 0.3858066630117833, 0.38572190201429807, 0.3635432287456949, 0.36351278362222283, 0.3635432287456949]

similarity3=[0.5778636637447012, 0.5288657114684102, 0.7505865952990389, 0.679542106957833, 0.6436508520286807, 0.679542106957833, 0.5483460386832888, 0.5483460386832888, 0.5483244494789303, 0.7704718372343412, 0.7704460845681002, 0.7114514054869984, 0.6325576995118218, 0.6325055302380096, 0.6325576995118218, 0.414218467100062, 0.6640661398728127, 0.5409185542065786, 0.6720836780494432, 0.6720735856117708, 0.6720195044491901, 0.6127659869831276, 0.9171258990300409, 0.6125051416064842, 0.38617919327804573, 0.38616453838135334, 0.38616453838135334, 0.6776448207915785, 0.6774544468701627, 0.6776448207915785, 0.6524793125515148, 0.6524793125515148, 0.6505302253966202, 0.4307649610668214, 0.4307649610668214, 0.5299917696601006, 0.34481826961991596, 0.3449026844302219, 0.34481826961991596, 0.5409736326405354, 0.5408581214383286, 0.5408675552746434, 0.25735798506004315, 0.25735798506004315, 0.25740455906370646, 0.47624672582462374, 0.47624672582462374, 0.47624672582462374, 0.3930446203143297, 0.39301777440782615, 0.39301777440782615, 0.557090942972094, 0.5570454022578315, 0.5772453993548398, 0.4485079138766565, 0.4485079138766565, 0.47506154443901544, 0.4334399678317037, 0.4334399678317037, 0.4334252189112176, 0.41317893804230205, 0.41317893804230205, 0.41317893804230205, 0.46969598025622333, 0.5449208303465614, 0.5449997886000024, 0.40548632929391404, 0.4970282185559906, 0.4970282185559906, 0.28218294778507513, 0.28218294778507513, 0.28218294778507513, 0.7990570154990193, 0.7990301388946973, 0.7990301388946973, 0.5130483000828613, 0.5130483000828613, 0.5131138850628956, 0.7255885938699029, 0.7256648965939693, 0.7256648965939693, 0.7185290119506106, 0.7185290119506106, 0.7185290119506106, 0.6946641376601989, 0.6953022099977036, 0.6946641376601989, 0.2667124786106884, 0.2667021408313746, 0.2668055258279286, 0.39870736531877793, 0.35314196562446726, 0.3986729482988898, 0.6429188955606232, 0.6428083737808097, 0.6095063321715317, 0.47691026331058317, 0.47697190561478203, 0.47691026331058317, 0.417379556202661, 0.46157274897942474, 0.5242472500157076, 0.46351202318495677, 0.46351202318495677, 0.46351202318495677, 0.58557187586642, 0.58557187586642, 0.5854939594757615, 0.5092528369782692, 0.5092528369782692, 0.5092528369782692, 0.7404215830750367, 0.8694092606356736, 0.8694092606356736, 0.23725992928251763, 0.2373017251284376, 0.2373395152809405, 0.8149869618463819, 0.8149869618463819, 0.8149869618463819, 0.8657080815203794, 0.9369673319166362, 0.8657080815203794, 0.3805374380176047, 0.36641605672488886, 0.36641605672488886, 0.5392093997243067, 0.5392093997243067, 0.5869211011380445, 0.3536987994290063, 0.3536987994290063, 0.35357590117844645, 0.5842487748233518, 0.6767207897245249, 0.5842487748233518, 0.5762827681974464, 0.5762827681974464, 0.5762827681974464, 0.8384815217604682, 0.8384815217604682, 0.8384815217604682, 0.7819107090333783, 0.8575444327174657, 0.7819486540111621, 0.4136796921561715, 0.41377001763415255, 0.41374766049197453, 0.3439123460363413, 0.3439123460363413, 0.3439123460363413, 0.8062754777540277, 0.8062754777540277, 0.8062870615221797, 0.6904757732600394, 0.6903388598181182, 0.7023493254416503, 0.8879926991317313, 0.8880398760549424, 0.8880398760549424, 0.3422956525409453, 0.3422956525409453, 0.3423464966458499, 0.8730179396188357, 0.8730489733201523, 0.8730347176085927, 0.6902239526782921, 0.6902239526782921, 0.6902239526782921, 0.5657278797861705, 0.5657193507809647, 0.5657193507809647, 0.8527690418801647, 0.8527690418801647, 0.8527690418801647, 0.3828513357473356, 0.3828513357473356, 0.38405652780516053, 0.38377099442860574, 0.3836636332675569, 0.3163059454093794, 0.2071106597824005, 0.20704985895654948, 0.20704985895654948, 0.4248504687858718, 0.4247564309672468, 0.4247564309672468, 0.34605683543463245, 0.37048023602627206, 0.3460022145615208, 0.48814310725734567, 0.5662033297113395, 0.48817151553652377, 0.3717235643101799, 0.37158770884013037, 0.37166395839971983, 0.33181086832705087, 0.3317678923614604, 0.3317678923614604, 0.29768904663996737, 0.29760426389670497, 0.29768904663996737, 0.31969505696804457, 0.31969505696804457, 0.31969505696804457, 0.6143805774511554, 0.6143577430185075, 0.6160613476825003, 0.41502221316017135, 0.41523315091362273, 0.41502221316017135, 0.3554673697559144, 0.3554461884755766, 0.3554673697559144]

similarity4=[0.7996178039719462, 0.7996235604519438, 0.7996235604519438, 0.6436508520286807, 0.679542106957833, 0.6436508520286807, 0.7320037325458781, 0.7320037325458781, 0.7320037325458781, 0.7115154397011639, 0.7114514054869984, 0.7114514054869984, 0.6564771219018858, 0.6195611824309806, 0.6564771219018858, 0.5409185542065786, 0.5409185542065786, 0.6939536245733913, 0.6795807740859883, 0.6796486575140407, 0.6796076304763963, 0.9171258990300409, 0.9171258990300409, 0.612620617315679, 0.20313083977553706, 0.3208506631680508, 0.3208506631680508, 0.7794989653981197, 0.7795192267816579, 0.7649170809251293, 0.6524793125515148, 0.6504668483546245, 0.6525446777159358, 0.4307729090334659, 0.5525842145325672, 0.5299917696601006, 0.34481826961991596, 0.3449026844302219, 0.34481826961991596, 0.47463767200835677, 0.4459158334549976, 0.4459158334549976, 0.25735798506004315, 0.25735798506004315, 0.25740455906370646, 0.24114346770797848, 0.26775656944858295, 0.26775656944858295, 0.39301777440782615, 0.48872364223878695, 0.39306210866100283, 0.5772453993548398, 0.5570454022578315, 0.557090942972094, 0.38144611408149326, 0.557809225939174, 0.5779192776006947, 0.35726532204282196, 0.35723224609372767, 0.5681735106693954, 0.41317893804230205, 0.41317893804230205, 0.24116236726877782, 0.2115945539073312, 0.2115945539073312, 0.29844246944636915, 0.5856682077186474, 0.47934288325742186, 0.3962591664944082, 0.28218294778507513, 0.28218294778507513, 0.22635945603263277, 0.9999988481092739, 1.0000000000000002, 0.2433398790108244, 0.6883347525012372, 0.7473739674968358, 0.3128370824761476, 0.7256648965939693, 0.7256648965939693, 0.24499570334188261, 0.7185290119506106, 0.7185290119506106, 0.5040466211395173, 0.7868334784371397, 0.7875603183730187, 0.1699766706049419, 0.26667093098174727, 0.2667021408313746, 0.6117501152154342, 0.42829135875632385, 0.5058670445650845, 0.15706870803437306, 0.6095693947541161, 0.642838133007802, 0.3700493565910048, 0.47691026331058317, 0.47697190561478203, 0.48057134445695565, 0.5000071258394309, 0.5000071258394309, 0.4311498535534553, 0.46351202318495677, 0.46351202318495677, 0.3956308710183561, 0.5880632322564892, 0.588022717344352, 0.3216653695321419, 0.7736734667830533, 0.7736734667830533, 0.2924321789723051, 0.7404215830750367, 0.8694092606356736, 0.2756253468070174, 0.28664975552354893, 0.30445971136266287, 0.46634990204565213, 0.7464019448671273, 0.7463725148463364, 0.46068080723455906, 0.6500383353623543, 0.8657080815203794, 0.15386844809976025, 0.36818186308539075, 0.21181141521725882, 0.46075322546182373, 0.5392093997243067, 0.5392093997243067, 0.2319507154359239, 0.3258342116272503, 0.3258342116272503, 0.45309729531887344, 0.34777867289549913, 0.3477622829198257, 0.7303546638186429, 0.5332372981172988, 0.5332372981172988, 0.34257959198165233, 0.6363427461310018, 0.4503668324761578, 0.2761825250493331, 0.7819107090333783, 0.7819486540111621, 0.2880296846869825, 0.39426312404259073, 0.39422533353550376, 0.30232552747292296, 0.33296845229741784, 0.29999394555258685, 0.2804620066292179, 0.8062754777540277, 0.8062754777540277, 0.3651027086080847, 0.5825433067342715, 0.5681005811711672, 0.27402620322886195, 0.7357555403309295, 0.8296916049990913, 0.23891567533089933, 0.3422956525409453, 0.3422140508571025, 0.17895441438393675, 0.4271521928344502, 0.8731207816122872, 0.35288908579206346, 0.6402869377846522, 0.650095419001682, 0.3063408726693606, 0.5657193507809647, 0.5907343930279394, 0.15598540689036158, 0.31813734142869177, 0.31813734142869177, 0.3178492487568545, 0.3784611241090053, 0.38402441601749604, 0.29956224657972613, 0.35820882180326863, 0.358128139481723, 0.24903681285567372, 0.606139589313755, 0.5483328600379044, 0.2494083666589625, 0.3627308807276851, 0.3627308807276851, 0.2614685568607895, 0.3460022145615208, 0.37044235748775217, 0.27965085485004154, 0.48814310725734567, 0.48811644564481355, 0.270714289815975, 0.3708752421102667, 0.3548131020552908, 0.34733487186790357, 0.33176599344548674, 0.3317678923614604, 0.48059125145868714, 0.29768904663996737, 0.29760426389670497, 0.13097831408376076, 0.2134181406633088, 0.2134181406633088, 0.15047240374899282, 0.6343406953564659, 0.6143577430185075, 0.3461291874226403, 0.38572190201429807, 0.3858066630117833, 0.379131535852987, 0.3635432287456949, 0.36351278362222283, 0.0]

In [ ]:
df_eval_rag['goal_sim']=pd.Series(similarity1)
df_eval_norag['goal_sim']=pd.Series(similarity2)
df_eval_raggoal['goal_sim']=pd.Series(similarity3)
df_eval_noraggoal['goal_sim']=pd.Series(similarity4)


def stat_goal(df, key:str):
    mean_a = df[key].mean()
    std_a = df[key].std()
    median_a = df[key].median()
    print(f"Mean of {key}: {mean_a:2f}")
    print(f"Standard Deviation of {key}: {std_a:2f}")
    print(f"Median of {key}: {median_a:2f}")

def plot_goal_vs_target_equal_ratio(df, a, b):
    fig = px.scatter(df, x=a, y=b, title=f'Scatter Plot of {a} vs {b}')
    fig.show()


stat_goal(df_eval_rag ,'goal_sim')
stat_goal( df_eval_norag,'goal_sim')
stat_goal( df_eval_raggoal,'goal_sim')
stat_goal( df_eval_noraggoal,'goal_sim')

plot_goal_vs_target_equal_ratio( df_eval_rag,'target_equal_ratio','goal_sim')
plot_goal_vs_target_equal_ratio( df_eval_norag,'target_equal_ratio','goal_sim')
plot_goal_vs_target_equal_ratio( df_eval_raggoal,'target_equal_ratio','goal_sim')
plot_goal_vs_target_equal_ratio( df_eval_noraggoal,'target_equal_ratio','goal_sim')

Mean of goal_sim: 0.525870
Standard Deviation of goal_sim: 0.183792
Median of goal_sim: 0.509119
Mean of goal_sim: 0.527884
Standard Deviation of goal_sim: 0.188357
Median of goal_sim: 0.530583
Mean of goal_sim: 0.535422
Standard Deviation of goal_sim: 0.183184
Median of goal_sim: 0.513114
Mean of goal_sim: 0.475869
Standard Deviation of goal_sim: 0.200216
Median of goal_sim: 0.445916
